<a href="https://colab.research.google.com/github/BaikaiL/voice/blob/main/translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 安装 Edge-TTS (用于高质量语音合成)
!pip install edge-tts nest_asyncio

In [3]:
import torch
import asyncio
import nest_asyncio
import edge_tts
from transformers import pipeline, AutoProcessor, AutoModelForSpeechSeq2Seq # 引入更底层的类
import IPython.display as ipd
import uuid
import gradio as gr

# 解决 Jupyter/Colab 中的异步循环冲突问题
nest_asyncio.apply()

class CantonesToMandarinTranslator:
    def __init__(self):
        # 自动检测 GPU
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

        print(f"正在初始化... (Device: {self.device})")

        # 1. 加载 ASR 模型
        print("正在加载 Whisper (ASR)...")

        # 1. 从仓库加载微调后的模型权重
        my_model_id = "baikai1022/whisper-small-cantonese-v2"
        model = AutoModelForSpeechSeq2Seq.from_pretrained(
            my_model_id,
            torch_dtype=self.torch_dtype,
            low_cpu_mem_usage=True,
            use_safetensors=True
        ).to(self.device)

        # 2. 从 OpenAI 原版加载处理器/分词器 (填补仓库缺失的文件)
        processor = AutoProcessor.from_pretrained("openai/whisper-small")

        # 3. 手动组装 Pipeline
        self.asr_pipe = pipeline(
            "automatic-speech-recognition",
            model=model,
            tokenizer=processor.tokenizer,           # 借用原版 tokenizer
            feature_extractor=processor.feature_extractor, # 借用原版 feature_extractor
            device=self.device,
            torch_dtype=self.torch_dtype,
        )

         # 2. 加载 NLLB 模型 (Translation)
        print("正在加载 NLLB (Translation)...")
        self.mt_pipe = pipeline(
            "translation",
            model="baikai1022/nllb-cantonese-mandarin-v1",
            device=self.device,
            torch_dtype=self.torch_dtype,
            src_lang="yue_Hant",
            tgt_lang="zho_Hans"
        )
        print("模型加载完成！")

    def run(self, audio_path, output_audio_path="output_mandarin.mp3"):
        print("\n" + "="*40)
        print(f"处理音频: {audio_path}")

        # --- 步骤 1: 语音识别 ---
        asr_result = self.asr_pipe(
            audio_path,
        )
        cantonese_text = asr_result["text"]
        print(f"[识别结果 (粤语)]: {cantonese_text}")

        # --- 步骤 2: 机器翻译 ---
        mt_result = self.mt_pipe(cantonese_text)
        mandarin_text = mt_result[0]['translation_text']
        print(f"[翻译结果 (国语)]: {mandarin_text}")

        # --- 步骤 3: 语音合成 ---
        print("正在合成语音...")
        voice = "zh-CN-XiaoxiaoNeural"

        async def _generate_tts():
            communicate = edge_tts.Communicate(mandarin_text, voice)
            await communicate.save(output_audio_path)

        asyncio.run(_generate_tts())

        print(f"合成完成: {output_audio_path}")
        print("="*40 + "\n")
        return output_audio_path, mandarin_text

    def process(self, audio_path):
        """
        专门给 Gradio 用的处理函数
        返回: (粤语文本, 普通话文本, 普通话音频路径)
        """
        if not audio_path:
            return "请先录音或上传文件", "", None

        print(f"正在处理: {audio_path}")

        # 1. ASR 识别
        # 指定 language="chinese" 以利用微调效果且避免报错
        asr_result = self.asr_pipe(audio_path)
        cantonese_text = asr_result["text"]

        # 2. MT 翻译
        mt_result = self.mt_pipe(cantonese_text)
        mandarin_text = mt_result[0]['translation_text']

        # 3. TTS 合成
        # 使用随机文件名，防止多人使用时冲突
        output_filename = f"out_{uuid.uuid4().hex[:8]}.mp3"
        voice = "zh-CN-XiaoxiaoNeural"

        async def _gen_tts():
            comm = edge_tts.Communicate(mandarin_text, voice)
            await comm.save(output_filename)

        asyncio.run(_gen_tts())

        return cantonese_text, mandarin_text, output_filename

# 运行测试
translator = CantonesToMandarinTranslator()



正在初始化... (Device: cuda)
正在加载 Whisper (ASR)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda


正在加载 NLLB (Translation)...


Device set to use cuda


模型加载完成！


In [3]:
# 请确保这里有文件，或者修改文件名
input_audio = "common_voice_zh-HK_22404635.wav"

import os
if os.path.exists(input_audio):
    output, text = translator.run(input_audio)
    ipd.display(ipd.Audio(output))
else:
    print(f"请上传一个名为 {input_audio} 的粤语录音文件来测试")


处理音频: common_voice_zh-HK_22404635.wav


`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536

[识别结果 (粤语)]: 黄师傅去驰云山、封华街帮个后生仔睇跌打。
[翻译结果 (国语)]: 黄师去云山、封华街帮个年轻人看跌打。
正在合成语音...
合成完成: output_mandarin.mp3



In [ ]:
def gradio_interface(audio):
    return translator.process(audio)

with gr.Blocks(title="粤语转普通话 AI") as demo:
    gr.Markdown("# 🇭🇰 粤语 -> 🇨🇳 普通话 智能转换器")
    gr.Markdown("基于 Whisper-Small (微调) + NLLB-200 (微调) + Edge-TTS")

    with gr.Row():
        # 左侧：输入区
        with gr.Column():
            # sources=["microphone"] 允许录音，type="filepath" 传递文件路径
            input_audio = gr.Audio(sources=["microphone", "upload"], type="filepath", label="请说粤语")
            btn = gr.Button("开始转换", variant="primary")

        # 右侧：输出区
        with gr.Column():
            txt_cantonese = gr.Textbox(label="识别结果 (粤语)")
            txt_mandarin = gr.Textbox(label="翻译结果 (普通话)")
            out_audio = gr.Audio(label="合成语音 (普通话)", autoplay=True)

    # 绑定点击事件
    btn.click(
        fn=gradio_interface,
        inputs=input_audio,
        outputs=[txt_cantonese, txt_mandarin, out_audio]
    )
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b6f2895b6f19cdb30f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


正在处理: /tmp/gradio/5230062fff037b6e20da8d64f6d6bd8253d740311d4ceb22ddcaa845846020bf/audio.wav


`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536

正在处理: /tmp/gradio/26182b77da921a2cda4d7747d3aaa50b63f9daab102938ffaaead44e417e6da0/audio.wav
正在处理: /tmp/gradio/a310effb49fb76be0284a5d2e9b56454d4276a6a928c7b490198526967c318e0/audio.wav
正在处理: /tmp/gradio/969dffe3f7714fee5bffc2a7e21333d2d096269064194809c1cc1b533cc17dd5/audio.wav
正在处理: /tmp/gradio/3aeb6e8491168ff30120b1946c5c0ad7159c15eae9061d32a2525d511f5ddf78/audio.wav
正在处理: /tmp/gradio/150948c5dd3dad8b617abb4d3f224c10a7a11d57f3bcc0042b789eabcf39f33c/audio.wav
正在处理: /tmp/gradio/bb9e63501d726bcbbea4bea8cc77038d2b3355d79211313f1851a47218313c1e/audio.wav


Your input_length: 334 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


...

Your input_length: 336 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1698, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/l

正在处理: /tmp/gradio/0f114c4fce6c68146cd6473c8c8278de8fa5e415ffd8d862ca2420d89a67a61a/audio.wav
正在处理: /tmp/gradio/96c6030b7bb6019a34b7fdb287c38ed448cfc4812687258e76323abd57a231d9/audio.wav
正在处理: /tmp/gradio/111db87c3220c696db1434619397ee43525392841779994c97cd1594ba4ab06a/audio.wav
正在处理: /tmp/gradio/7714e39770fd752ac1cc204e1ce9fea81c4bea820609c6a2321a56baccb93654/audio.wav
正在处理: /tmp/gradio/45554502f13a2909a89c76ecef6d70b13dffee400c977043db7d9b77495ad074/audio.wav
正在处理: /tmp/gradio/4b0231300f835f96e7728b00fa38a81a32ba21a32f3690bbae5d479ccd78d9b0/audio.wav


Your input_length: 337 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


正在处理: /tmp/gradio/4b0231300f835f96e7728b00fa38a81a32ba21a32f3690bbae5d479ccd78d9b0/audio.wav


Your input_length: 337 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
